# Assignment 2 Part 2

Abhinav Kumar
10/13/2025

PatchCore on MVTec-AD

In [29]:
from pathlib import Path
import time, numpy as np, pandas as pd

from lightning import Trainer, seed_everything
from anomalib.data import MVTecAD
from anomalib.models import Patchcore
from anomalib.visualization import ImageVisualizer

seed_everything(42)


Seed set to 42


42

In [30]:
def build_dm(category: str,
             root: str = "./data",
             train_batch_size: int = 8,
             eval_batch_size: int = 8,
             num_workers: int = 4):
    return MVTecAD(
        root=root,
        category=category,
        train_batch_size=train_batch_size,
        eval_batch_size=eval_batch_size,
        num_workers=num_workers,
    )

In [ ]:
# --- speed knobs ---
MAX_EPOCHS = 1
LIMIT_TRAIN = 0.05 
LIMIT_VAL = 0.0 
LIMIT_TEST = 0.2
NUM_SANITY_VAL_STEPS = 0
BATCH_SIZE = 8
NUM_WORKERS = 4
# --------------------

CATEGORIES = ("tile", "leather", "grid")

OUTDIR = Path("./outputs_hertz"); OUTDIR.mkdir(parents=True, exist_ok=True)

def run_category_fast(category: str):
    dm = build_dm(category,
                  train_batch_size=BATCH_SIZE,
                  eval_batch_size=BATCH_SIZE,
                  num_workers=NUM_WORKERS)
    model = Patchcore()

    trainer = Trainer(
        accelerator="cpu", devices=1, precision="32-true",
        max_epochs=MAX_EPOCHS,
        limit_train_batches=LIMIT_TRAIN,
        limit_val_batches=LIMIT_VAL,
        limit_test_batches=LIMIT_TEST,
        num_sanity_val_steps=NUM_SANITY_VAL_STEPS,
        logger=False, enable_checkpointing=True,
        default_root_dir=str(OUTDIR / category),
        log_every_n_steps=5,
        enable_progress_bar=True,
    )

    t0 = time.time()
    trainer.fit(model=model, datamodule=dm)
    metrics_list = trainer.test(model=model, datamodule=dm, verbose=False)
    dt = time.time() - t0

    metrics = metrics_list[0] if isinstance(metrics_list, list) else metrics_list
    img_auc = metrics.get("image_auroc", metrics.get("AUROCImage", np.nan))
    pxl_auc = metrics.get("pixel_auroc", metrics.get("AUROCPixel", np.nan))

    print(f"[{category}] done in {dt/60:.1f} min | image_auroc={img_auc} | pixel_auroc={pxl_auc}")
    return float(img_auc or np.nan), float(pxl_auc or np.nan)

rows = []
for cat in CATEGORIES:
    ia, pa = run_category_fast(cat)
    rows.append({"category": cat, "image_auroc": ia, "pixel_auroc": pa})
    df = pd.DataFrame(rows).set_index("category")
    display(df)
    df.to_csv(OUTDIR / "auroc_summary.partial.csv")

df = pd.DataFrame(rows).set_index("category")
df.to_csv(OUTDIR / "auroc_summary.csv")
display(df)
print(f"\nMean Image AUROC: {df['image_auroc'].mean():.4f}")
print(f"Mean Pixel AUROC: {df['pixel_auroc'].mean():.4f}")
print("\nArtifacts saved to:", OUTDIR.resolve())